In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path 
import os 
from gpt3forchem.data import discretize, get_opv_data

from gpt3forchem.api_wrappers import query_gpt3, fine_tune, extract_prediction

from gpt3forchem.baselines import XGBClassificationBaseline

from gpt3forchem.input import create_single_property_forward_prompts

from sklearn.model_selection import train_test_split

from pycm import ConfusionMatrix

import time

In [ ]:
data = get_opv_data()

/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/gpt3forchem/data.py:88: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(datadir, "opv.csv"))


In [ ]:
data

,SMILES,selfies,ID,PCE_max(%),PCE_ave(%),Voc(V),Jsc(mA\tcm^2),FF,Mw(kgmol^-1),Monomer(gmol^-1),...,ecpf_1055,ecpf_1056,ecpf_1057,ecpf_1058,ecpf_1059,ecpf_1060,ecpf_1061,ecpf_1062,ecpf_1063,pce_bin
0,C/C=C/C1=CC(N(C2=CC=C(OCCCCCCCC)C=C2)C3=C4C=CC...,[C][/C][=C][/C][=C][C][Branch2][=Branch1][#C][...,28.0,0.30,0.30,0.55,0.859,0.484,7.8,840.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C/C=C/C1=CC(N(C2=CC=C(OCCCCCCCC)C=C2)C3=C4C=CC...,[C][/C][=C][/C][=C][C][Branch2][Branch2][Branc...,29.0,0.20,0.20,0.50,0.761,0.432,8.2,1003.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,C/C=C/C1=CC(N(C2=CC=C(OCCCCCCCC)C=C2)C3=C4C=CC...,[C][/C][=C][/C][=C][C][Branch2][Ring2][=C][N][...,27.0,0.40,0.40,0.70,1.076,0.445,5.7,533.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,C/C=C/C1=CC(N(C2=CC=C(OCCCCCCCC)C=C2)C3=C4C=CC...,[C][/C][=C][/C][=C][C][Branch2][Branch2][=C][N...,30.0,0.80,0.80,0.80,1.558,0.546,4.3,1061.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CC(C1=C2N=C(C3=CC=C(OCCCCCCCC)C=C3)C(C4=CC=C(O...,[C][C][Branch2][Branch1][=Branch1][C][=C][N][=...,603.0,1.90,1.90,0.77,5.000,0.503,660.9,1011.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,O=C1N(CCCCCCCC)C(C2=CC=C(C)S2)=C3C1=C(C4=CC=C(...,[O][=C][N][Branch1][=Branch2][C][C][C][C][C][C...,51.0,3.20,3.20,0.80,8.600,0.470,63.0,956.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1094,O=C1N(CCCCCCCC)C(C2=CC=C(C)S2)=C3C1=C(C4=CC=C(...,[O][=C][N][Branch1][=Branch2][C][C][C][C][C][C...,57.0,0.88,0.88,0.74,2.510,0.470,27.1,941.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1095,O=C1N(CCCCCCCCCC)C(C2=CC=C(C)S2)=C3C1=C(C4=CC=...,[O][=C][N][Branch1][O][C][C][C][C][C][C][C][C]...,52.0,3.80,3.80,0.77,9.100,0.550,56.7,1068.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1096,O=C1N(C[C@@H](CCCC)CC)C(C2=CC=C(C)S2)=C3C1=C(C...,[O][=C][N][Branch1][O][C][C@@H1][Branch1][Bran...,58.0,1.12,1.12,0.44,4.470,0.570,14.5,968.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
discretize(data, "PCE_ave(%)")

In [ ]:
train, test = train_test_split(data, train_size=.8, random_state=42, stratify=data["PCE_ave(%)_cat"])

In [ ]:
FEATURES = [f for f in data.columns if f.startswith("ecpf_")]

## Train baseline model

In [ ]:
baseline = XGBClassificationBaseline(None)
baseline.fit(train[FEATURES], train["PCE_ave(%)_cat"])

In [ ]:
baseline_predictions = baseline.predict(test[FEATURES])

In [ ]:
cm = ConfusionMatrix(actual_vector=test["PCE_ave(%)_cat"].to_numpy(), predict_vector=baseline_predictions)

In [ ]:
print(cm)

Predict          large            medium           small            very large       very small       
Actual
large            13               20               4                0                0                

medium           10               23               26               2                4                

small            4                12               34               0                13               

very large       2                2                2                3                0                

very small       1                9                16               0                20               





Overall Statistics : 

95% CI                                                            (0.35745,0.48801)
ACC Macro                                                         0.76909
ARI                                                               0.08518
AUNP                                                              0.60839
AUNU                                 

### Now, let's try to fine-tune GPT-3 on this data

In [ ]:
train_prompts = create_single_property_forward_prompts(train, "PCE_ave(%)_cat", {
    "PCE_ave(%)_cat": "photoconversion efficiency",
}, representation_col='SMILES'
)

In [ ]:
test_prompts = create_single_property_forward_prompts(test, "PCE_ave(%)_cat", {
    "PCE_ave(%)_cat": "photoconversion efficiency",
}, representation_col='SMILES'
)

In [ ]:
train_size = len(train_prompts)
test_size = len(test_prompts)

print(train_size, test_size)
filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
train_filename = (
    f"run_files/{filename_base}_train_prompts_pce_{train_size}.jsonl"
)
valid_filename = (
    f"run_files/{filename_base}_valid_prompts_pce_{test_size}.jsonl"
)

train_prompts.to_json(train_filename, orient="records", lines=True)
test_prompts.to_json(valid_filename, orient="records", lines=True)

878 220


In [ ]:
fine_tune(train_filename, valid_filename, "ada")

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

'ada:ft-lsmoepfl-2022-09-12-06-56-57'

In [ ]:
completions = query_gpt3('ada:ft-lsmoepfl-2022-09-12-06-56-57', test_prompts)

In [ ]:
predictions = [int(extract_prediction(completions, i)) for i in range(len(completions['choices']))]

In [ ]:
true = [int(prompt.split('@')[0]) for prompt in  test_prompts['completion']]

In [ ]:
cm_gpt3 = ConfusionMatrix(true, predictions)

In [ ]:
print(cm_gpt3)

Predict  0        1        2        3        4        
Actual
0        19       13       13       1        0        

1        8        25       27       3        0        

2        3        18       36       8        0        

3        0        9        19       9        0        

4        0        1        3        5        0        





Overall Statistics : 

95% CI                                                            (0.33969,0.4694)
ACC Macro                                                         0.76182
ARI                                                               0.05016
AUNP                                                              0.59137
AUNU                                                              0.57897
Bangdiwala B                                                      0.18361
Bennett S                                                         0.25568
CBA                                                               0.28048
CSI                           